In [1]:
import keras
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import merge
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import AveragePooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K


Using TensorFlow backend.


In [5]:
img_size = 250
batch_size = 32
epochs = 200
num_classes = 250
K.set_image_dim_ordering('th')

train_path = 'C:/Users/rantw/Documents/School/CS5590/Project/training/'
val_path = 'C:/Users/rantw/Documents/School/CS5590/Project/validation/'

In [6]:
train_data = ImageDataGenerator(rescale=1./255,
                                featurewise_center=True,  # set input mean to 0 over the dataset
                                samplewise_center=True,  # set each sample mean to 0
                                featurewise_std_normalization=True,  # divide inputs by std of the dataset
                                samplewise_std_normalization=True,  # divide each input by its std
                                rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
                                width_shift_range=0.4,  # randomly shift images horizontally (fraction of total width)
                                height_shift_range=0.4,  # randomly shift images vertically (fraction of total height)
                                zoom_range=0.4, #zooms the image randomly 
                                horizontal_flip=True,  # randomly flip images
                                vertical_flip=True)  # randomly flip images)

val_data = ImageDataGenerator(rescale=1./255)

train_gen = train_data.flow_from_directory(train_path,
                                    target_size=(img_size, img_size),
                                    batch_size=batch_size,
                                    color_mode='grayscale',
                                    class_mode='categorical')

val_gen = val_data.flow_from_directory(val_path,
                                    target_size=(img_size, img_size),
                                    batch_size=batch_size,
                                    color_mode='grayscale',
                                    class_mode='categorical')

Found 15000 images belonging to 250 classes.
Found 5000 images belonging to 250 classes.


In [8]:
print('Training samples:', train_gen.samples)
print('Validation samples:',val_gen.samples)
val_ind = val_gen.class_indices
train_ind = train_gen.class_indices
#print('Training Classes:', train_gen.class_indices)
#print('Validation Classes:', val_gen.class_indices)
print('Validation Folder:', set(val_ind) - set(train_ind)) #if set() then no missing or mispelled folders
print('Training Folder:', set(train_ind) - set(val_ind)) #if set() then no missing or mispelled folders

Training samples: 15000
Validation samples: 5000
Validation Folder: set()
Training Folder: set()


In [18]:
input_shape = Input(shape=(1, img_size, img_size))

L1_conv = Conv2D(64, (15, 15), strides=(3, 3),  activation='relu')(input_shape)
L1_pool = MaxPooling2D(pool_size=(3, 3),strides=(2, 2))(L1_conv)

L2_conv = Conv2D(128, (5, 5), strides=(1, 1), activation='relu')(L1_pool)
L2_pool = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(L2_conv)

L3_conv = Conv2D(256, (5, 5), strides=(1, 1), activation='relu')(L2_pool)
L3_pool = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(L3_conv)

# Tower A
TowerA_conv1 = Conv2D(48, (1, 1))(L3_pool)
TowerA_conv2 = Conv2D(64, (3, 3))(TowerA_conv1)
TowerA_conv3 = Conv2D(128, (1, 1))(TowerA_conv2)

# Tower B
TowerB_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(L3_pool)
TowerB_conv1 = Conv2D(64, (1, 1))(TowerB_pool)
TowerB_conv2 = Conv2D(128, (1, 1))(TowerB_conv1)

# Tower C
TowerC_conv1 = Conv2D(64, (3, 3))(L3_pool)
TowerC_conv2 = Conv2D(128, (1, 1))(TowerC_conv1)

# Merge Towers
L_merged = keras.layers.concatenate([TowerA_conv3, TowerB_conv2, TowerC_conv2], axis=1)

L5_pool = MaxPooling2D(pool_size=(3, 3),strides=(2, 2))(L_merged)

L6_conv = Conv2D(256, (1, 1), strides=(1, 1), activation='relu')(L5_pool)

L_flat = Flatten()(L6_conv)
L6_fullC1 = Dense(256, activation='relu')(L_flat)
L_dr = Dropout(0.5)(L6_fullC1)
L7_fullC2 = Dense(num_classes, activation='softmax')(L_dr)
model = Model(input_shape, L7_fullC2)


lrate = 0.000001
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1, 250, 250)  0                                            
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 64, 79, 79)   14464       input_6[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_21 (MaxPooling2D) (None, 64, 39, 39)   0           conv2d_56[0][0]                  
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 128, 35, 35)  204928      max_pooling2d_21[0][0]           
__________________________________________________________________________________________________
max_poolin

In [19]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor = 'val_acc', patience=30),
            ModelCheckpoint(filepath = 'bestModel.h5', monitor ='val_acc', save_best_only=True)]

history = model.fit_generator(train_gen,
                              steps_per_epoch=train_gen.samples // batch_size,
                              validation_data=val_gen,
                              validation_steps=val_gen.samples // batch_size,
                              epochs=epochs,
                              verbose=1,
                              callbacks = callbacks)

Epoch 1/200
468/468 [==============================] - 805s 2s/step - loss: 5.5530 - acc: 0.0043 - val_loss: 5.5233 - val_acc: 0.0034
Epoch 2/200
468/468 [==============================] - 774s 2s/step - loss: 5.5491 - acc: 0.0032 - val_loss: 5.5221 - val_acc: 0.0038
Epoch 3/200
468/468 [==============================] - 768s 2s/step - loss: 5.5500 - acc: 0.0040 - val_loss: 5.5228 - val_acc: 0.0052
Epoch 4/200
468/468 [==============================] - 760s 2s/step - loss: 5.5425 - acc: 0.0045 - val_loss: 5.5224 - val_acc: 0.0028
Epoch 5/200
468/468 [==============================] - 760s 2s/step - loss: 5.5482 - acc: 0.0044 - val_loss: 5.5230 - val_acc: 0.0048
Epoch 6/200
468/468 [==============================] - 758s 2s/step - loss: 5.5431 - acc: 0.0044 - val_loss: 5.5225 - val_acc: 0.0044
Epoch 7/200
468/468 [==============================] - 754s 2s/step - loss: 5.5446 - acc: 0.0042 - val_loss: 5.5230 - val_acc: 0.0030
Epoch 8/200
468/468 [==============================] - 757s 2s

KeyboardInterrupt: 